# Graph Convolutional Network with Numpy



### Introduction and libraries

We are going to build a Graph Convolutional Network (GCN) similar to the one described by Kipf and Welling (2016) and train it in a node classification task using the Cora dataset (Sen et al., 2008). We are going to implement this from scratch, without using deep learning frameworks such as Tensorflow or Pytorch.

Therefore, the only libraries we need are numpy (for mathematical operations) and spektral (for downloading the dataset).

In [1]:
#UNCOMMENT TO INSTALL 

#!pip install spektral

import numpy as np
import spektral

The first thing to do is to download the dataset and extract the variables we need for building and training the model. These are:

-`train_mask`, `val_mask` and `test_mask`: binary vectors indicating which nodes are to be included in the training, validation and test set respectively.

-`features`: a matrix which includes features vector corresponding to each node. Each node represents a scientific article, and each feature indicates whether a certain key word is present in that specific paper. 

-`adj`: the adjacency matrix specifying the nodes' connections. These indicate the citation network, i.e. which paper was cited in which other paper.

-`labels`: The labels of each node, corresponding to the topic of the scientific paper. Labels are one-hot encoded.

In [2]:
cora_dataset = spektral.datasets.citation.Citation(name='cora')

train_mask = cora_dataset.mask_tr
val_mask = cora_dataset.mask_va
test_mask = cora_dataset.mask_te

graph = cora_dataset.graphs[0]
features = graph.x
adj = graph.a
labels = graph.y


C:\Users\franc\anaconda3\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


### Data exploration and preprocessing

Now we can do a little data exploration.

In [3]:
print('Size of the training set:', np.sum(train_mask))
print('Size of the validation set:', np.sum(val_mask))
print('Size of the test set:', np.sum(test_mask))

print('Size of the feature matrix:', features.shape)
print('Number of labels:', labels.shape[1])

print('Feature value range:', np.min(features), '-', np.max(features))

print('Adjacency matrix type:', type(adj))


Size of the training set: 140
Size of the validation set: 500
Size of the test set: 1000
Size of the feature matrix: (2708, 1433)
Number of labels: 7
Feature value range: 0.0 - 1.0
Adjacency matrix type: <class 'scipy.sparse.csr.csr_matrix'>


It looks like we have a very small dataset for training (140 nodes). There are 7 possible labels, and the feature values range from 0 to 1, suggesting they are categorical. Let's quickly verify this.


In [4]:
print('Feature unique values:', np.unique(features))

Feature unique values: [0. 1.]


It seems to indeed be the case.

We also have a feature space of size 1433, and 2708 total nodes. This means our adjacency matrix should be of size 2703x2703. However, here this is encoded as a sparse matrix, and a dense one would suit our purpose better, as it can be easily used for matrix operations when implementing the model. Let's convert it to a dense representation.

In [5]:
adj = adj.todense()

### Loss and activation functions 

Let's now define a few useful functions.

-`softmax`: the softmax activation function, which we are going to need to apply to the last layer of our network. It will allow us to have probabilities that sum up to 1 as an output.

-`relu`: the relu activation function, to be used in hidden layers. It is a very common way to add some non-linearity to a model.

-`masked_loss`: a categorical crossentropy loss with a mask applied to it, so that we can make sure to train and evaluate the model on the right data. The mask needs to be proprocessed to enter the function, but we'll see that later.

-`masked_accuracy`: a function to calculate accuracy with a mask applied top it, so that we evaluate our model with the right dataset. As above, the mask needs preprocessing.

In [5]:
def softmax(logits):
    num = np.exp(logits)
    den = np.sum(np.exp(logits), axis = -1)
    den = np.reshape(den, (den.shape[0], 1))
    return num/den

def relu(logits):
    ispos = logits>0
    ispos = ispos.astype('float32')
    return np.multiply(logits, ispos)

def masked_loss(logits, labels, mask):
    loss = -np.multiply(labels, np.log(softmax(logits)))
    loss = np.multiply(loss, mask)
    loss = np.sum(loss, axis=1)
    return np.mean(loss)

def masked_accuracy(logits, labels, mask):    
    pred = logits.argmax(1)
    real = np.argmax(labels, axis=-1)
    real = np.reshape(real, pred.shape)
    is_correct = np.equal(pred, real)
    is_correct = np.multiply(is_correct, mask)
    return np.mean(is_correct)

### Adam optimizer

As we are not allowed to use Tensorflow or some other deep learning framework, we have to implement our optimizer from scratch. As in the original paper (Kipf & Welling, 2016), we use an Adam optimizer (Kingma & Ba, 2014), a very common choice.

Here we use a standard hyperparameter setting (default in Tensorflow).

In [6]:
class Adam():
    def __init__(self, n_layers=2, eta=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.m_dW = []
        self.v_dW = []
        self.m_db = []
        self.v_db = []
        for l in range(n_layers):
            self.m_dW.append(0)
            self.v_dW.append(0)
            self.m_db.append(0)
            self.v_db.append(0)
            
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.eta = eta
    def update(self, t, W, b, dW, db):
        for l in range(len(W)):
            ## momentum beta 1
            # *** weights *** #
            self.m_dW[l] = self.beta1*self.m_dW[l] + (1-self.beta1)*dW[l]
            # *** biases *** #
            self.m_db[l] = self.beta1*self.m_db[l] + (1-self.beta1)*db[l]

            ## rms beta 2
            # *** weights *** #
            self.v_dW[l] = self.beta2*self.v_dW[l] + (1-self.beta2)*np.power(dW[l], 2)
            # *** biases *** #
            self.v_db[l] = self.beta2*self.v_db[l] + (1-self.beta2)*np.power(db[l], 2)

            ## bias correction
            m_dW_corr = self.m_dW[l]/(1-np.power(self.beta1, t))
            m_db_corr = self.m_db[l]/(1-np.power(self.beta1, t))
            v_dW_corr = self.v_dW[l]/(1-np.power(self.beta2, t))
            v_db_corr = self.v_db[l]/(1-np.power(self.beta2, t))

            ## update weights and biases
            W[l] = W[l] - self.eta*(m_dW_corr/(np.sqrt(v_dW_corr)+self.epsilon))
            b[l] = b[l] - self.eta*(m_db_corr/(np.sqrt(v_db_corr)+self.epsilon))
        return W, b

### Some building blocks
We can now start building the model itself. First of all we are going to define two functions:

-`gnn_layer`: a function defining a forward pass in a single Graph Neural Network (GNN) layer. It uses matrix operations for to perform message passing, which makes the process more efficient (the alternative being nested for loops, which are both slower and harder to read). It takes as input the nodes `features`, an adjacency matrix `adj`, weigth matrix `W` and bias vector `b`, as well as some `activation` function. It returns all the processing stages of the forward pass, which we are going to need for the backpropagation. This is a very general-purpose function, and, as we'll see later, can be used for models other than GCNs.

-`gnn_forward`: a function performing the whole forward pass for an arbitrary number of layers. The inputs `features` and `adj` are the same as above, however, `W` and `b` are now lists, with as many entries as there are layers in the network. `n_units` is also a list, specifying how many units there are in each layer (including the output layer). For all layers but the output one the function will apply the `relu` activation and an additional dropout layer with a rate specified by the parameter `dropout`. The final layer has no dropout and has a `softmax` activation function applied to it to output label probabilities.

In [7]:
def gnn_layer(features, adj, W, b, activation):
    h = np.dot(features, W) + b 
    z = np.dot(adj, h) 
    a = activation(z)
    return h, z, a

def gnn_forward(features, n_units, adj, W, b, dropout):
    H = []
    Z = []
    A = []
    
    for l, n in enumerate(n_units):

        if l == 0: #First layer
            h, z, a = gnn_layer(features, adj, W[l], b[l], relu)           
        elif l < len(n_units)-1: #Middle layers
            h, z, a = gnn_layer(a, adj, W[l], b[l], relu)
        else: #Output layer
            h, z, a = gnn_layer(a, adj, W[l], b[l], softmax)
        
        #Dropout
        if dropout > 0 and l < len(n_units)-1:
            p = np.random.uniform(size=(1, h.shape[1]))
            keep = p > dropout
            h = np.multiply(h, keep)
            z = np.multiply(z, keep)
            a = np.multiply(a, keep)

        H.append(h)
        Z.append(z)
        A.append(a)
    
    return H, Z, A


### The model itself
We are building a `GNN` class, so that we can later build different instances of it and play around with hyperparameters. 

The method `__init__`, called to initialise an instance of the class, takes as input the number of node features `n_features`, a list containing the number of units in each layer `n_units` and a `dropout` layer to be applied only during training to prevent overfitting. All the weights and biases are initialised by sampling from a normal distribution with mean 0 and standard deviation 0.01, and stored as lists (`W` and `b` respectively).

The method `fit` trains the model, taking as input the nodes' `features` and `labels`, the number of epochs `n_epochs` we want to train our model for, an `optimizer` for updating the weights and the training and validation masks (`train_mask` and `val_mask`) to train and evaluate the model on the nodes corresponding to the trainin and validation datasets, respectively.
The masks are divided by their mean value, making them easier to use for calculating the loss and accuracy (see above). The method loops over epochs. For each iteration, it performs a forward pass to calculate outputs and a backwards pass (back propagation) to calculate the gradients of the loss with respect to the trainable parameters contained in the lists `W` and `b` (i.e. weights and biases). It then applies the Adam optimizer (defined above) to update such parameters. Finally, it evaluates the loss and accuracy for both the training and validation nodes, storing the weigths and biases in the lists `W_best` and `b_best` only when the validation loss is lower than its previous minimum value. 

Finally, the method `evaluate` uses `W_best` and `b_best` to evaluate the model performance on a certain set of nodes determined by the input parameter `mask`, returning the value of the `loss` and the `accuracy`.

In [10]:
class GNN():
    def __init__(self, n_features, adj, n_units, dropout):
        
        self.adj = adj
        self.n_units = n_units
        self.dropout = dropout
        
        #Initialise all weights as an empty list
        W = []
        b = []
        for l, n in enumerate(n_units):
            
            #Get dimensions for weight matrix 
            if l == 0:
                dims = (n_features, n) #Original features
            else:
                dims = (n_units[l-1], n)
            
            #Initialise weight matrix (random normal initalisation)
            W.append(np.random.normal(0, 0.01, dims))
            b.append(np.random.normal(0, 0.01, (1, n)))
         
        self.W = W
        self.b = b
        
        self.best_W = W
        self.best_b = b
        
    def fit(self, features, labels, n_epochs, optimizer, 
           train_mask, val_mask):
        
        #Preprocess the masks for easier use 
        train_mask = train_mask.astype('float32')
        train_mask /= np.mean(train_mask)
        train_mask = train_mask.reshape(train_mask.shape[0], 1)
        
        val_mask = val_mask.astype('float32')
        val_mask /= np.mean(val_mask)
        val_mask = val_mask.reshape(val_mask.shape[0], 1)
        
        #Initialise best val loss to large number
        best_val_loss = 1e10 
        
        for e in range(n_epochs):
            
            #Forward prop
            H, Z, A = gnn_forward(features, self.n_units, self.adj, 
                                  self.W, self.b, dropout=self.dropout)

            #Backprop
            dW = []
            db = []
            dH = []
            counter = -1

            for l in range(len(self.n_units)-1, -1, -1):

                counter +=1
                if l == len(self.n_units)-1:
                    dz = np.multiply((A[l] - labels), train_mask) #Mask to take only the gradient for training examples
                    dh = np.dot(self.adj.T, dz) 
                    dw = np.dot(A[l-1].T, dh)
                elif l==0:
                    da = np.dot(dH[counter-1], self.W[l+1].T)
                    dz = np.multiply(da, Z[l]>0)
                    dh = np.dot(self.adj.T, dz)
                    dw = np.dot(features.T, dh)
                else:
                    da = np.dot(dH[counter-1], self.W[l+1].T)
                    dz = np.multiply(da, Z[l]>0)
                    dh = np.dot(self.adj, dz)
                    dw = np.dot(A[l-1].T, dh)
                    
                dH.append(dh)
                dW.append(dw)
                db.append(np.mean(dh, axis=0))

            dW.reverse()
            db.reverse()

            self.W, self.b = optimizer.update(e+1, self.W, self.b, 
                                                        dW, db)
            
            #Forward pass for calculating losses and accuracies
            _, _, A = gnn_forward(features, self.n_units, self.adj, 
                                  self.W, self.b, dropout=0)
            
            #Calculate losses and accuracies
            train_loss = masked_loss(A[-1], labels, train_mask)
            val_loss = masked_loss(A[-1], labels, val_mask)
            
            train_accuracy = masked_accuracy(A[-1], labels, train_mask)
            val_accuracy = masked_accuracy(A[-1], labels, val_mask)
            
            print('Epoch ', e+1, ': \n'
                 'Training loss: ', train_loss, ' | ', 
                 'Validation loss: ', val_loss, ' | \n',
                 'Training accuracy: ', train_accuracy, ' | ', 
                 'Validation accuracy: ', val_accuracy, ' | ')
            
            #Update best weights only if validation loss has improved
            if val_loss < best_val_loss:
                self.best_W = self.W
                self.best_b = self.b
    
    def evaluate(self, features, labels, mask):
        
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        mask = mask.reshape(mask.shape[0], 1)
        
        _, _, A = gnn_forward(features, self.n_units, self.adj, 
                           self.best_W, self.best_b, dropout=0)
        
        loss = masked_loss(A[-1], labels, mask)
        accuracy = masked_accuracy(A[-1], labels, mask)
        
        return loss, accuracy


### Building a Graph Convolutional Network

Before we can create an instance of our model, we have to go through one final step, modifying the adjacency matrix as in the paper we are using as a guide (Kipf & Welling, 2016).

We can then finally build A GCN. We are using a 64-dimensional node representation. We are also using a dropout rate of 0.5, as in the original paper (Kipf & Welling, 2016). 

In [11]:
#Normalise the adjecency matrix as in Kipf & Welling (2017)
adj += np.eye(adj.shape[0])
D = np.sum(adj, axis=-1)
I = np.eye(adj.shape[0])
D_norm = np.multiply(I, (1/np.sqrt(D)))
adj_norm = np.dot(D_norm, np.dot(adj, D_norm))

GCN = GNN(features.shape[1], adj_norm, [64, 7], 0.5)

### Training and testing
We can now create an instance of our optimiser (with default settings) and `fit` our model to the training set. We are training for 100 epochs, but the model is going to take much less to converge.

In [12]:
optimizer = Adam(n_layers = 2)

GCN.fit(features, labels, 100, optimizer, 
           train_mask, val_mask)

Epoch  1 : 
Training loss:  1.9445570730261017  |  Validation loss:  1.9448956901334866  | 
 Training accuracy:  0.14285715  |  Validation accuracy:  0.162  | 
Epoch  2 : 
Training loss:  1.941682052896121  |  Validation loss:  1.9437417170029123  | 
 Training accuracy:  0.42142856  |  Validation accuracy:  0.26599997  | 
Epoch  3 : 
Training loss:  1.9364550846000994  |  Validation loss:  1.941491951596453  | 
 Training accuracy:  0.5  |  Validation accuracy:  0.27199998  | 
Epoch  4 : 
Training loss:  1.928627213174946  |  Validation loss:  1.938123359756131  | 
 Training accuracy:  0.60714287  |  Validation accuracy:  0.274  | 
Epoch  5 : 
Training loss:  1.9173838089983026  |  Validation loss:  1.9332656569670923  | 
 Training accuracy:  0.73571426  |  Validation accuracy:  0.32999998  | 
Epoch  6 : 
Training loss:  1.9012335643424556  |  Validation loss:  1.9261958728313  | 
 Training accuracy:  0.7928571  |  Validation accuracy:  0.39599997  | 
Epoch  7 : 
Training loss:  1.87894

Now we have the weights and biases from the epochs with the smallest validation loss stored  as `W_best` and `b_best`. We can use these to evaluate our model on the training, validation and test nodes.

In [15]:
loss_train, accuracy_train = GCN.evaluate(features, labels, train_mask)
loss_val, accuracy_val = GCN.evaluate(features, labels, val_mask)
loss_test, accuracy_test = GCN.evaluate(features, labels, test_mask)

print(' Training loss: ', loss_train, ' | ', 'Training accuracy: ', accuracy_train, '\n', 
      'Validation loss: ', loss_val, ' | ', 'Validation accuracy: ', accuracy_val, '\n',
      'Test loss: ', loss_test, ' | ', 'Test accuracy: ', accuracy_test)
                  
                 

 Training loss:  1.1654487838774377  |  Training accuracy:  1.0 
 Validation loss:  1.405015515133568  |  Validation accuracy:  0.75200003 
 Test loss:  1.3868402164352012  |  Test accuracy:  0.7809999


Our test accuracy should be between 75% and 80% (there are fluctuations due to the random weight initialisation), which is pretty close with that reported in the original paper (Kipf & Welling, 2017) for the Cora dataset (81.5%). Not bad!

### Comparison with a Multi Layer Perceptron (MLP)
How much did the graph structural information encoded in the adjacency matrix influence our results? We can verifying this by training a Multi Layer Perceptron (MLP) on the nodes' features only, neglecting all the edges. To implement this in practice, we can build another instance of `GNN`, using an identity matrix instead of the adjacency one. This will make every node only connected to itself, eliminating all the structural information encoded in the edges.

For the sake of a fair comparison, let's keep all the hyper-parameters the same.

In [16]:
MLP = GNN(features.shape[1], np.eye(adj.shape[0]), [64, 7], 0.5)

In [17]:
optimizer = Adam(n_layers = 2, eta = 0.01)

MLP.fit(features, labels, 100, optimizer, 
           train_mask, val_mask)

Epoch  1 : 
Training loss:  1.944341985449011  |  Validation loss:  1.9456767244795672  | 
 Training accuracy:  0.23005907  |  Validation accuracy:  0.23005912  | 
Epoch  2 : 
Training loss:  1.940832193728074  |  Validation loss:  1.9447602466898972  | 
 Training accuracy:  0.41728202  |  Validation accuracy:  0.41728222  | 
Epoch  3 : 
Training loss:  1.9349723555483016  |  Validation loss:  1.9433287092345144  | 
 Training accuracy:  0.45051703  |  Validation accuracy:  0.4505169  | 
Epoch  4 : 
Training loss:  1.9257890948183713  |  Validation loss:  1.9414754122133324  | 
 Training accuracy:  0.47562775  |  Validation accuracy:  0.47562793  | 
Epoch  5 : 
Training loss:  1.9122526715481416  |  Validation loss:  1.9386857765417886  | 
 Training accuracy:  0.5110783  |  Validation accuracy:  0.51107824  | 
Epoch  6 : 
Training loss:  1.8913800972008359  |  Validation loss:  1.9345680697630423  | 
 Training accuracy:  0.52954215  |  Validation accuracy:  0.52954185  | 
Epoch  7 : 
Tr

Now let's test it as before:

In [19]:
loss_train, accuracy_train = MLP.evaluate(features, labels, train_mask)
loss_val, accuracy_val = MLP.evaluate(features, labels, val_mask)
loss_test, accuracy_test = MLP.evaluate(features, labels, test_mask)

print(' Training loss: ', loss_train, ' | ', 'Training accuracy: ', accuracy_train, '\n', 
      'Validation loss: ', loss_val, ' | ', 'Validation accuracy: ', accuracy_val, '\n',
      'Test loss: ', loss_test, ' | ', 'Test accuracy: ', accuracy_test)

 Training loss:  1.1654254565721214  |  Training accuracy:  0.5387741 
 Validation loss:  1.652390808995176  |  Validation accuracy:  0.5387742 
 Test loss:  1.6609423662965361  |  Test accuracy:  0.5387737


Quite a big drop in performance for all node sets. We can conclude that the structural information encoded in the edges (and thus the adjacency matrix) was crucial for the success of our GCN. The MLP, on the other hand, is completely oblivious of how the nodes it is being trained on are connected with each other, and with less information comes worse accuracy. 

### Scaling up the model
A bigger model can usually capture more complex relationships amongst features and between features and labels. However, they increase the chances of overfitting (which is a real risk with a small trining dataset like ours). Let's test this by scaling up the model in two ways:

-Adding units: we'll call this instance `GCN_wide`

-Adding a layer: we'll call this instance `GCN_deep`

In [23]:
GCN_wide = GNN(features.shape[1], adj_norm, [256, 7], 0.5)

optimizer = Adam(n_layers = 2)

GCN_wide.fit(features, labels, 100, optimizer, 
           train_mask, val_mask)

Epoch  1 : 
Training loss:  1.9403334540476433  |  Validation loss:  1.9435950244965312  | 
 Training accuracy:  0.74285716  |  Validation accuracy:  0.36599997  | 
Epoch  2 : 
Training loss:  1.9279835219417687  |  Validation loss:  1.9370433325381418  | 
 Training accuracy:  0.92142856  |  Validation accuracy:  0.59199995  | 
Epoch  3 : 
Training loss:  1.9046159308855608  |  Validation loss:  1.9246803561936017  | 
 Training accuracy:  0.9571429  |  Validation accuracy:  0.686  | 
Epoch  4 : 
Training loss:  1.8668441461575238  |  Validation loss:  1.9053950171398848  | 
 Training accuracy:  0.97142863  |  Validation accuracy:  0.71999997  | 
Epoch  5 : 
Training loss:  1.8056552766042377  |  Validation loss:  1.8763497849978366  | 
 Training accuracy:  0.97857153  |  Validation accuracy:  0.758  | 
Epoch  6 : 
Training loss:  1.719511615011205  |  Validation loss:  1.8363795501501317  | 
 Training accuracy:  0.97857153  |  Validation accuracy:  0.77  | 
Epoch  7 : 
Training loss:  

In [24]:
loss_train, accuracy_train = GCN_wide.evaluate(features, labels, train_mask)
loss_val, accuracy_val = GCN_wide.evaluate(features, labels, val_mask)
loss_test, accuracy_test = GCN_wide.evaluate(features, labels, test_mask)

print(' Training loss: ', loss_train, ' | ', 'Training accuracy: ', accuracy_train, '\n', 
      'Validation loss: ', loss_val, ' | ', 'Validation accuracy: ', accuracy_val, '\n',
      'Test loss: ', loss_test, ' | ', 'Test accuracy: ', accuracy_test)
                  

 Training loss:  1.1654223079428223  |  Training accuracy:  1.0 
 Validation loss:  1.4000471078002905  |  Validation accuracy:  0.76399994 
 Test loss:  1.3774315441639304  |  Test accuracy:  0.78699994


We observe that adding hidden units made very little difference. Let's try adding layers:

In [29]:
GCN_deep = GNN(features.shape[1], adj_norm, [256, 64, 7], 0.5)

optimizer = Adam(n_layers = 3)

GCN_deep.fit(features, labels, 100, optimizer, 
           train_mask, val_mask)

Epoch  1 : 
Training loss:  1.9457447059766688  |  Validation loss:  1.9463379127316653  | 
 Training accuracy:  0.14285715  |  Validation accuracy:  0.058  | 
Epoch  2 : 
Training loss:  1.945044971173609  |  Validation loss:  1.94628358712265  | 
 Training accuracy:  0.19999999  |  Validation accuracy:  0.094  | 
Epoch  3 : 
Training loss:  1.942894483201862  |  Validation loss:  1.9452578742518623  | 
 Training accuracy:  0.40714285  |  Validation accuracy:  0.21799998  | 
Epoch  4 : 
Training loss:  1.9388657232655477  |  Validation loss:  1.9429934689187005  | 
 Training accuracy:  0.39999998  |  Validation accuracy:  0.27199998  | 
Epoch  5 : 
Training loss:  1.9308096415435314  |  Validation loss:  1.9383401606754547  | 
 Training accuracy:  0.4214286  |  Validation accuracy:  0.29  | 
Epoch  6 : 
Training loss:  1.9171176317332617  |  Validation loss:  1.9301181192502503  | 
 Training accuracy:  0.5642857  |  Validation accuracy:  0.35599998  | 
Epoch  7 : 
Training loss:  1.89

In [30]:
loss_train, accuracy_train = GCN_deep.evaluate(features, labels, train_mask)
loss_val, accuracy_val = GCN_deep.evaluate(features, labels, val_mask)
loss_test, accuracy_test = GCN_deep.evaluate(features, labels, test_mask)

print(' Training loss: ', loss_train, ' | ', 'Training accuracy: ', accuracy_train, '\n', 
      'Validation loss: ', loss_val, ' | ', 'Validation accuracy: ', accuracy_val, '\n',
      'Test loss: ', loss_test, ' | ', 'Test accuracy: ', accuracy_test)

 Training loss:  1.1654231496776957  |  Training accuracy:  1.0 
 Validation loss:  1.4147507217815862  |  Validation accuracy:  0.75399995 
 Test loss:  1.3996099499461363  |  Test accuracy:  0.7649999


Again, very little difference in terms of performance. The only things worth noting about `GCN_wide` and `GCN_deep` are the following:

-The models take longer to train. Of course this is expected, as bigger matrices entail a bigger computational cost.

-The models take longer to converge. Again, no surprise here, as the optimizer has to navigate a higher-dimensional space to minimise the loss.

-There is no major increase in overfitting. This is somewhat surprising, but it is most likely due to the dropout layers. In fact, even when I tried the original `GCN` model without dropout, it would overfit much more, with validation and test accuracies never exceeding 55% (while the training accuracy got to 100% very quickly). I did not include this experiment for brevity. 

We can thus conclude that there is no real benefit in increasing the size of the model in our case.

### Conclusions
In this notebook we implemented a GCN model from scratch, using numpy only. We further veryfied the importance of the graph structure encoded in the adjacency matrix by comparing our model with a simple MLP.


### References
Kingma, D. P., & Ba, J. (2014). Adam: A method for stochastic optimization. arXiv preprint arXiv:1412.6980.

Kipf, T. N., & Welling, M. (2016). Semi-supervised classification with graph convolutional networks. arXiv preprint arXiv:1609.02907.

Sen, P., Namata, G., Bilgic, M., Getoor, L., Galligher, B., & Eliassi-Rad, T. (2008). Collective classification in network data. AI magazine, 29(3), 93-93.
